In [ ]:
import open3d as o3d
import pandas as pd
import numpy as np
from collections import namedtuple
from scipy.spatial import cKDTree
import time


In [ ]:
#%% KD-Tree Experiments

# Params class containing the counter and the main LineSet object holding all line subsets
class params():

    counter = 0
    full_line_set = o3d.geometry.LineSet()

# Callback function for generating the LineSets from each neighborhood
def build_edges(vis):
    """Build edges function."""
    # Run this part for each point in the point cloud
    if params.counter < len(points):
        # Find the K-nearest neighbors for the current point. In our case we use 6
        [k, idx, _] = pcd_tree.search_knn_vector_3d(points[params.counter,:], 20)
        # Get the neighbor points from the indices
        points_temp = points[idx,:]
        
        # Create the neighbours indices for the edge array
        neighbours_num = np.arange(len(points_temp))
        # Create a temp array for the center point indices
        point_temp_num = np.zeros(len(points_temp))
        # Create the edges array as a stack from the current point index array and the neighbor indices array
        edges = np.vstack((point_temp_num,neighbours_num)).T

        # Create a LineSet object and give it the points as nodes together with the edges
        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector(points_temp)
        line_set.lines = o3d.utility.Vector2iVector(edges)
        # Color the lines by either using red color for easier visualization or with the colors from the point cloud
        line_set.paint_uniform_color([1, 0, 0])
        # line_set.paint_uniform_color(colors[params.counter,:])
        
        # Add the current LineSet to the main LineSet
        params.full_line_set+=line_set
        
        # if the counter just started add the LineSet geometry
        if params.counter==0:
            vis.add_geometry(params.full_line_set)
        # else update the geometry 
        else:
            vis.update_geometry(params.full_line_set)
        # update the render and counter
        vis.update_renderer()
        params.counter +=1
    else:
        # if the all point have been used reset the counter and clear the lines
        params.counter=0
        params.full_line_set.clear()

#KD Tree Visualizer

# Load point cloud .ply into Open3D point cloud object
point_cloud = o3d.io.read_point_cloud('../DATA/structure_verviers.ply') 

# Downsample the point cloud using voxel downsampling
point_cloud = point_cloud.voxel_down_sample(voxel_size=0.2)

# get the points and colors as separate numpy arrays
points = np.asarray(point_cloud.points)
colors = np.asarray(point_cloud.colors)

# Calculate the KDTree from the point cloud
pcd_tree = o3d.geometry.KDTreeFlann(point_cloud)

# Recolor the point cloud in blue, just to get a better contrast with the distance lines
# point_cloud.paint_uniform_color([0, 0, 1])

# Initialize a visualizer object
vis = o3d.visualization.Visualizer()
# Create a window, name it and scale it
vis.create_window(window_name='3D Data Visualize', width=800, height=600)

# Set background color to black
opt = vis.get_render_option()
opt.background_color = np.asarray([0, 0, 0])

# Add the point cloud to the visualizer
vis.add_geometry(point_cloud)

# Register a new animation callback function
vis.register_animation_callback(build_edges)

# Run the visualizater
vis.run()
# Once the visualizer is closed destroy the window and clean up
vis.destroy_window()

In [4]:
#%% 2. Octree Visualizer

curr_max_depth = 8

point_cloud = o3d.io.read_point_cloud('../DATA/structure_verviers.ply') 

octree = o3d.geometry.Octree(max_depth = curr_max_depth)
octree.convert_from_point_cloud(point_cloud)
o3d.visualization.draw_geometries([octree])

In [5]:
octree.locate_leaf_node(point_cloud.points[0])    

(OctreePointColorLeafNode with color [0.572549, 0.545098, 0.419608] containing 77 points.,
 OctreeNodeInfo with origin [6.12408, -6.94074, 0.625316], size 0.0905313, depth 8, child_index 6)

In [6]:
pcd = o3d.io.read_point_cloud("../DATA/Seychelles-beach.ply")
o3d.visualization.draw_geometries([pcd])

In [ ]:
# Generate random 3D point cloud
num_points = 10000
points = np.random.rand(num_points, 3)

# KD-Tree (using scipy)
start_time = time.time()
kdtree = cKDTree(points)
kd_build_time = time.time() - start_time

# Octree (using Open3D)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

start_time = time.time()
octree = o3d.geometry.Octree(max_depth=8)
octree.convert_from_point_cloud(pcd, size_expand=0.01)
oct_build_time = time.time() - start_time

print(f"KD-Tree build time: {kd_build_time:.6f} seconds")
print(f"Octree build time: {oct_build_time:.6f} seconds")

In [ ]:
#BVH Visualization

Node = namedtuple("Node", ["left", "right", "bbox", "points"])

def create_bbox(points):
    """Create bbox function."""
    return np.min(points, axis=0), np.max(points, axis=0)

def split_points(points, axis):
    """Split points function."""
    median = np.median(points[:, axis])
    left_mask = points[:, axis] < median
    return points[left_mask], points[~left_mask]

def build_bvh(points, depth=0, max_depth=10, min_points=5):
    """Build bvh function."""
    if len(points) <= min_points or depth >= max_depth:
        return Node(None, None, create_bbox(points), points)
    
    axis = depth % 3
    left_points, right_points = split_points(points, axis)
    
    left_node = build_bvh(left_points, depth + 1, max_depth, min_points)
    right_node = build_bvh(right_points, depth + 1, max_depth, min_points)
    
    bbox = (np.minimum(left_node.bbox[0], right_node.bbox[0]),
            np.maximum(left_node.bbox[1], right_node.bbox[1]))
    
    return Node(left_node, right_node, bbox, None)

root = build_bvh(np.array(pcd.points))

print("BVH created successfully!")
print(f"Root bounding box: {root.bbox}")


#%% Visualization Definition

def create_bbox_lines(bbox):
    """Create bbox lines function."""
    min_point, max_point = bbox
    points = [
        min_point,
        [max_point[0], min_point[1], min_point[2]],
        [max_point[0], max_point[1], min_point[2]],
        [min_point[0], max_point[1], min_point[2]],
        [min_point[0], min_point[1], max_point[2]],
        [max_point[0], min_point[1], max_point[2]],
        max_point,
        [min_point[0], max_point[1], max_point[2]]
    ]
    lines = [
        [0, 1], [1, 2], [2, 3], [3, 0],
        [4, 5], [5, 6], [6, 7], [7, 4],
        [0, 4], [1, 5], [2, 6], [3, 7]
    ]
    colors = [[1, 0, 0] for _ in range(len(lines))]
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(points),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

def visualize_bvh(root, max_depth=3):
    """Visualize bvh function."""
    geometries = []

    def add_bbox(node, depth=0):
        """Add bbox function."""
        if depth > max_depth:
            return
        geometries.append(create_bbox_lines(node.bbox))
        if node.left:
            add_bbox(node.left, depth + 1)
        if node.right:
            add_bbox(node.right, depth + 1)

    add_bbox(root)
    return geometries


#%% Create Open3D point cloud

# Visualize BVH
bvh_geometries = visualize_bvh(root)

# Visualize
o3d.visualization.draw_geometries([pcd] + bvh_geometries)

In [30]:
# Memory
def process_in_memory(dataset):
    """Process in memory function."""
    # Load the entire dataset into memory
    entire_dataset = np.loadtxt(dataset, delimiter=';', skiprows=1)
    
    # Now you can perform computations on the entire dataset
    result = np.mean(entire_dataset, axis=0)
    
    return result

# Out-of-Core
def process_out_of_core(dataset, chunk_size=1000):
    """Process out of core function."""
    # Read the dataset in chunks
    reader = pd.read_csv(dataset, delimiter=';', chunksize=chunk_size)
    
    # Initialize an empty array to accumulate results
    result_accumulator = np.zeros((1, 6))  

    counter=1
    for chunk in reader:
        # Perform computations on each chunk
        result_chunk = np.mean(chunk.to_numpy(), axis=0)
        
        # Accumulate results
        result_accumulator += result_chunk
        counter+=1
    # return result_accumulator / len(reader)
    return result_accumulator/counter

dataset = '../DATA/bike_florent.txt'

t0 = time.time()
inm = process_in_memory(dataset)
t1 = time.time()
outm = process_out_of_core(dataset)
t2 = time.time()

print(t1-t0, 'in memory results: ', inm)
print(t2-t1, 'out of core results: ', outm)

0.24349474906921387 in memory results:  [  1.45135827   1.25249302   0.45861719 100.01257653  92.40968783
  78.16664096]
0.7775394916534424 out of core results:  [[ 1.44960817  1.24867481  0.45720447 99.75950647 92.17670988 77.9702425 ]]
